All the packages that will be used on this dataset are imported.

In [1]:
import pandas as pd
import numpy as np
import sklearn as skl
import matplotlib as plt
import scipy as sp
import seaborn as sbs
from sklearn.linear_model import Lasso
from sklearn.cross_validation import train_test_split
import random
import datetime

C:\Users\musicalmichi\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Cleaning

The data we are using comes from three files: airlines.csv, airports.csv, and flights.csv. The file airlines.csv identifies each airline in the dataset with a two-letter acronym corresponding to the airline variable in flights.csv. The file airports.csv includes information on each airport referenced in flights.csv, including the airport name and the airport's location. Flights.csv is the main dataset we will be using. It contains 31 variables and has 5,819,079 entries with data about all the flights in 2015. The data files were obtained from __[Kaggle](https://www.kaggle.com/usdot/flight-delays/data)__.

In [2]:
airlines = pd.read_csv('airlines.csv' , low_memory=False)
airports = pd.read_csv('airports.csv' , low_memory=False)
flights = pd.read_csv('flights.csv' , low_memory=False)

We have decided to use only a subset of the data from the flights dataset, because of the large number of entries. We will be focusing on the following airlines: United Air Lines Inc., American Airlines Inc., Frontier Airlines Inc., JetBlue Airways, Skywest Airlines Inc. Alaska Airlines Inc., Spirit Air Lines, Southwest Airlines Co., and Delta Air Lines Inc.

The three date variables YEAR, MONTH, and DAY from the flights dataset are consolidated into a single datetime variable, DATE.

In [3]:
excludedAirlines = ["VX","MQ","HA","EV","US"]
flights = flights[~flights['AIRLINE'].isin(excludedAirlines)]

flights['DATE'] = pd.to_datetime(flights[['YEAR','MONTH', 'DAY']])

The function format_hour(), defined below, converts a string to a datetime variable, which is much more convenient to work with.

In [4]:
# Function that converts the 'HHMM' string to datetime.time
def format_hour(nums):
    if pd.isnull(nums):
        return np.nan
    else:
        if nums == 2400: nums = 0
        nums = "{0:04d}".format(int(nums))
        hour = datetime.time(int(nums[0:2]), int(nums[2:4]))
        return hour

The function format_hour() is called on by three time variables in the flights dataset: DEPARTURE_TIME, SCHEDULED_ARRIVAL, and ARRIVAL_TIME. Now the date and time variables are all datetime objects.

In [5]:
flights['DEPARTURE_TIME'] = flights['DEPARTURE_TIME'].apply(format_hour)
flights['SCHEDULED_ARRIVAL'] = flights['SCHEDULED_ARRIVAL'].apply(format_hour)
flights['ARRIVAL_TIME'] = flights['ARRIVAL_TIME'].apply(format_hour)

flights.loc[:5, ['SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL', 'DEPARTURE_TIME',
             'ARRIVAL_TIME', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY']]

,SCHEDULED_DEPARTURE,SCHEDULED_ARRIVAL,DEPARTURE_TIME,ARRIVAL_TIME,DEPARTURE_DELAY,ARRIVAL_DELAY
0,5,04:30:00,23:54:00,04:08:00,-11.0,-22.0
1,10,07:50:00,00:02:00,07:41:00,-8.0,-9.0
3,20,08:05:00,00:15:00,07:56:00,-5.0,-9.0
4,25,03:20:00,00:24:00,02:59:00,-1.0,-21.0
5,25,06:02:00,00:20:00,06:10:00,-5.0,8.0


The YEAR, MONTH, and DAY variables are removed form the flights dataset as they are no longer needed due to the new DATE variable. The following variables are also removed from the flights dataset because they do not provide any information that would help answer our questions of interest: TAXI_OUT, TAXI_IN, WHEELS_ON, WHEELS_OFF, and DIVERTED.

In [6]:
variables_to_remove = ['TAXI_OUT', 'TAXI_IN', 'WHEELS_ON', 'WHEELS_OFF', 'YEAR', 
                       'MONTH','DAY', 'DIVERTED']
flights.drop(variables_to_remove, axis = 1, inplace = True)

The flights data is split into training and test data. We have set a random seed to ensure reproducibility.

In [7]:
random.seed(1313)
train_flights, test_flights = train_test_split(flights,test_size=0.2)